# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [45]:
# import packages
import pandas as pd
import numpy as np
from scipy import stats

# read stata file in
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [46]:
# number of callbacks for black-sounding names
print(' Number of callbacks for black-sounding names: ', sum(data[data.race=='b'].call))

 Number of callbacks for black-sounding names:  157.0


In [47]:
# display the first five rows of data
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [48]:
# display information about columns
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [49]:
# display shape of data
data.shape

(4870, 65)

In [50]:
# extract 'call' column of black-sounding names and white-sounding names
black_call = data[data['race'] == 'b']['call']
white_call = data[data['race'] == 'w']['call']

# calculate number of samples
n_black = len(black_call)
n_white = len(white_call)

# proportion that get callback
black_prop_call = np.sum(black_call) / n_black
white_prop_call = np.sum(white_call) / n_white

# print proportions that get call backs
print('Given this sample of 4,870 resumes, approximately', round(black_prop_call,3) * 100, 
      '% of people with black-sounding names receive call backs while approximately', 
      round(white_prop_call,3) * 100, '% of people with white-sounding names receive call backs')

Given this sample of 4,870 resumes, approximately 6.4 % of people with black-sounding names receive call backs while approximately 9.7 % of people with white-sounding names receive call backs


# 1. What test is appropriate for this problem? Does CLT apply?

We are testing whether the average rate of callbacks differs significantly between resumes with black-sounding names and resumes with white-sounding names. There are 4,870 observations in our data set, which means that the CLT applies to our analysis (> 30 observations) and it will likely follow a normal distribution. Since we are comparing the means of two samples but we don't know the population standard deviation, we will perform a two sided $t$ test.

# 2. What are the null and alternate hypotheses?

$H_0$: There is no significant difference between the callback rates of resumes with white-sounding names and resumes with black-sounding names

$H_A$: There is a significant difference between the callback rates of resumes with white-sounding names and resumes with black-sounding names

# 3. Compute margin of error, confidence interval, and p-value.

We will be calculating a 95% confidence interval to determine if the difference in the proportion of callbacks for people with white-sounding names and the proportion of callbacks for people with black-sounding names is statistically significant. To calculate the $t$ statistic, we will use this formula:

$t$ = Y$_1$ − Y$_2$ / sqrt( s$_1$$^2$ / N$_1$ + s$_2$$^2$ / N$_2$ )

where:
* Y$_1$ and Y$_2$ are the two sample means
* s$_1$$^2$ and s$_2$$^2$ are the two sample variances
* N$_1$ and N$_2$ are the two sample sizes

In [51]:
# calculate standard error
SE = np.sqrt(np.std(black_call) ** 2 / n_black + np.std(white_call) ** 2 / n_white)

# calculate and print difference of means
diff_means = abs(np.mean(black_call) - np.mean(white_call))
print('The difference in the means of our two samples is approximately', round(diff_means,3))

# calculate t_critical
t_critical = stats.t.ppf(0.975,df=n_black-1)

# calculate and print margin of error
margin_error = t_critical * SE
print('The margin of error is approximately', round(margin_error,3))

# calculate and print 95% confidence interval
conf_int = [diff_means - margin_error, diff_means + margin_error]
print('95% confidence interval: ', conf_int)

The difference in the means of our two samples is approximately 0.032
The margin of error is approximately 0.015
95% confidence interval:  [0.016770261163944672, 0.047295448546176483]


In [52]:
# calculate and print t score
t = diff_means / SE
print('t = ', round(t,3))

# calcualte and print p value for two sided test
p_t = stats.t.sf(t,df=n_black-1)*2
print('p = p( t <= ', -round(t,3), ') + p( t >= ', round(t,3), ') = ', round(p_t,3))
print('p (unrounded) is ', p_t)

t =  4.116
p = p( t <=  -4.116 ) + p( t >=  4.116 ) =  0.0
p (unrounded) is  3.9905077647e-05


Our p value is very small, so we reject the null hypothesis. There is a significant difference between callback rates for resumes with black-sounding names and resumes with white-sounding names.

# 4. Write a story describing the statistical significance in the context or the original problem.

Although we've made strides to reduce the presence of racism in our society, it's still quite prevalent in cultures around the world. In this study, researchers studied whether candidates with 'black-sounding' names and 'white-sounding' names were equally treated in being called back for interviews. All the resumes were identical, but it turns out that resumes with black-sounding names had a callback rate of approximately 6.4% while resumes with white-sounding names had a callback rate of approximately 9.7%. We performed a two sample t-test to determine whether this difference in callback rates was statistically significant. We ended up with a very low p value, concluding that the difference is indeed statistically significant. This suggests that racism really is still pervasive, even in the United States.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The only variable I studied in my analysis is race/name, so I can only confirm that race/name has a significant impact on whether a candidate will receive a callback for an interview. Further testing on other variables needs to be completed in order to determine whether those other factors significantly effect callback rates.